In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

import transformer_model

In [9]:
df = pd.read_csv("cgm_data.csv").sample(frac=1)
for column in df.columns:
    df[column] = df[column].astype(float)
df.rename({0:"ID"}, axis=1)
df = df.drop("ptid",axis=1)

In [5]:
df.shape

(97, 480)

In [6]:
df.head()

,0 days 00:00:00,0 days 00:15:00,0 days 00:30:00,0 days 00:45:00,0 days 01:00:00,0 days 01:15:00,0 days 01:30:00,0 days 01:45:00,0 days 02:00:00,0 days 02:15:00,...,4 days 21:30:00,4 days 21:45:00,4 days 22:00:00,4 days 22:15:00,4 days 22:30:00,4 days 22:45:00,4 days 23:00:00,4 days 23:15:00,4 days 23:30:00,4 days 23:45:00
8,165.6,162.0,176.4,205.2,234.0,259.2,280.8,291.6,297.0,304.2,...,167.4,160.2,154.8,156.6,154.8,149.4,147.6,142.2,140.4,149.4
62,131.4,124.2,118.8,111.6,106.2,102.6,100.8,109.8,120.6,122.4,...,167.4,162.0,162.0,171.0,171.0,167.4,160.2,149.4,151.2,160.2
83,61.2,57.6,54.0,66.6,88.2,106.2,120.6,117.0,100.8,91.8,...,118.8,120.6,108.0,99.0,95.4,84.6,73.8,68.4,73.8,82.8
39,149.4,165.6,176.4,181.8,181.8,180.0,178.2,176.4,165.6,153.0,...,117.0,122.4,142.2,172.8,190.8,181.8,163.8,154.8,147.6,124.2
48,199.8,190.8,183.6,167.4,145.8,133.2,129.6,129.6,151.2,180.0,...,158.4,160.2,163.8,158.4,153.0,149.4,147.6,145.8,140.4,140.4


In [11]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cpu


In [1]:
# Assuming input sequence length is 100, feature size is 480 (glucose level at 480 time points)
# Output sequence length is 1 (glucose level at next time point)
model = transformer_model.TransformerModel(feature_size=480, hidden_dim=512, output_dim=1, nhead=4, num_layers=2).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


NameError: name 'transformer_model' is not defined

In [ ]:
# Assuming you've defined `train_dataloader` and `val_dataloader`
# that loads your training and validation data respectively

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    epoch_loss = 0
    for batch in dataloader:
        # Assuming each batch in your dataloader is a tuple (inputs, targets)
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

def validate_one_epoch(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

# Actual training loop
n_epochs = 10
for epoch in range(n_epochs):
    train_loss = train_one_epoch(model, train_dataloader, criterion, optimizer, device)
    val_loss = validate_one_epoch(model, val_dataloader, criterion, device)
    print(f"Epoch {epoch+1}: Train Loss: {train_loss}, Validation Loss: {val_loss}")
